In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torchvision import datasets, transforms, models

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization

2023-06-02 20:59:39.851278: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
DATA_DIR = "../sample"
BATCH_SIZE = 8

# Preparing the data

In [2]:
# image_files = glob.glob(DATA_DIR + "/*/image*.npy")
# mask_files = glob.glob(DATA_DIR + "/*/label*.npy")
# image_files.sort()
# mask_files.sort()
# #create a dataframe
# df = pd.DataFrame({"image": image_files, "mask": mask_files})
# df.head()

In [3]:
# df.to_csv("sample.csv", index=False)

# Pytorch

In [46]:
#create a dataloader class

class DataLoader():
    def __init__(self, metadata_dir, batch_size):
        self.metadata_dir = metadata_dir
        self.batch_size = batch_size
        df = pd.read_csv(metadata_dir + "/sample.csv")
        self.df = df.sample(frac=1).reset_index(drop=True)

    def __iter__(self):
        for i in range(0, len(self.df), self.batch_size):
            batch_df = self.df[i:i+self.batch_size]
            image_files = list(batch_df["image"])
            mask_files = list(batch_df["mask"])
            images = np.stack([np.load(f) for f in image_files])
            masks = np.stack([np.load(f) for f in mask_files])
            #make the images from hwc to chw
            images = images.transpose(0, 3, 1, 2)
            yield torch.from_numpy(images).float(), torch.from_numpy(masks).float()

    def __len__(self):
        return len(self.all_image_files)//batch_size

In [47]:
data = DataLoader(".", batch_size=BATCH_SIZE)

In [48]:
for image, label in data:
    print(image.shape, label.shape)
    break

torch.Size([8, 3, 256, 256]) torch.Size([8, 256, 256, 1])


In [44]:
class UNetEncoder(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=padding)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.batchnorm(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.batchnorm(x)
        x = self.relu(x)
        return x

class UNetDecoder(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=padding)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x, skip_x):
        x = F.interpolate(x, scale_factor=2, mode="bilinear", align_corners=True)
        x = torch.cat([x, skip_x], axis=1)
        x = self.conv1(x)
        x = self.batchnorm(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.batchnorm(x)
        x = self.relu(x)
        return x

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.enc1 = UNetEncoder(in_channels, 64)
        self.enc2 = UNetEncoder(64, 128)
        self.enc3 = UNetEncoder(128, 256)
        self.enc4 = UNetEncoder(256, 512)
        self.enc5 = UNetEncoder(512, 1024)

        self.dec1 = UNetDecoder(1024, 512)
        self.dec2 = UNetDecoder(512, 256)
        self.dec3 = UNetDecoder(256, 128)
        self.dec4 = UNetDecoder(128, 64)

        self.out = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        #encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(F.max_pool2d(enc1, kernel_size=2))
        enc3 = self.enc3(F.max_pool2d(enc2, kernel_size=2))
        enc4 = self.enc4(F.max_pool2d(enc3, kernel_size=2))
        enc5 = self.enc5(F.max_pool2d(enc4, kernel_size=2))

        #decoder
        dec1 = self.dec1(enc5, enc4)
        dec2 = self.dec2(dec1, enc3)
        dec3 = self.dec3(dec2, enc2)
        dec4 = self.dec4(dec3, enc1)

        out = self.out(dec4)
        return out

In [45]:
unet = UNet(3, 1)
# unet
#fit the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
unet.to(device)
optimizer = torch.optim.Adam(unet.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

def iou(pred, target):
    pred = torch.sigmoid(pred)
    pred = (pred > 0.5).float()
    intersection = (pred * target).sum()
    union = (pred + target).sum()
    return (intersection + 1e-6) / (union + 1e-6)

epochs = 10
for epoch in range(epochs):
    unet.train()
    epoch_loss = 0
    epoch_iou = 0
    for image, mask in data:
        image = image.to(device)
        mask = mask.to(device)
        pred_mask = unet(image)
        loss = criterion(pred_mask, mask)
        iou_score = iou(pred_mask, mask)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_iou += iou_score.item()
    print(f"Epoch {epoch+1} Loss: {epoch_loss/len(data)}, IoU: {epoch_iou/len(data)}")

RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[8, 256, 256, 3] to have 3 channels, but got 256 channels instead

In [65]:
#create a dataloader class

class DataLoader(tf.data.Dataset):
    def __init__(self, metadata_dir, batch_size):
        self.metadata_dir = metadata_dir
        self.batch_size = batch_size
        df = pd.read_csv(metadata_dir + "/sample.csv")
        self.df = df.sample(frac=1).reset_index(drop=True)

    def __getitem__(self):
        for i in range(0, len(self.df), self.batch_size):
            batch_df = self.df[i:i+self.batch_size]
            image_files = list(batch_df["image"])
            mask_files = list(batch_df["mask"])
            images = np.stack([np.load(f) for f in image_files])
            masks = np.stack([np.load(f) for f in mask_files])
            #make the images from hwc to chw
            # images = images.transpose(0, 3, 1, 2)
            return tf.convert_to_tensor(images, dtype=tf.float32), tf.convert_to_tensor(masks, dtype=tf.float32)
    def __len__(self):
        return len(self.all_image_files)//batch_size

In [66]:
data = DataLoader(".", batch_size=4)

for image, mask in data:
    print(image.shape, mask.shape)
    break

TypeError: Can't instantiate abstract class DataLoader with abstract methods _inputs, element_spec

# Tensorflow

In [3]:
df = pd.read_csv("sample.csv")
df = df.sample(frac=1).reset_index(drop=True)

def create_dataset():
    for i in range(0, len(df), BATCH_SIZE):
        batch_df = df[i : i + BATCH_SIZE]
        image_files = list(batch_df["image"])
        mask_files = list(batch_df["mask"])
        images = np.stack([np.load(f) for f in image_files])
        masks = np.stack([np.load(f) for f in mask_files])
        # make the images from hwc to chw
        # images = images.transpose(0, 3, 1, 2)
        yield tf.convert_to_tensor(images, dtype=tf.float32), tf.convert_to_tensor(masks, dtype=tf.float32)

data = tf.data.Dataset.from_generator(create_dataset, (tf.float32, tf.float32))

2023-06-02 20:59:44.632251: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
class UNetEncoder(tf.keras.layers.Layer):
    def __init__(
        self, filters, kernel_size=(3, 3), activation="relu", padding="same", **kwargs
    ):
        super(UNetEncoder, self).__init__(**kwargs)
        self.conv1 = tf.keras.layers.Conv2D(
            filters, kernel_size, activation=activation, padding=padding
        )
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(
            filters, kernel_size, activation=activation, padding=padding
        )
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))

    def call(self, inputs):
        # print(inputs.shape)
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        pool = self.pool(x)
        return x, pool

In [5]:
class UNetDecoder(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size=(3, 3), activation='relu', padding='same', **kwargs):
        super(UNetDecoder, self).__init__(**kwargs)
        self.up_conv = tf.keras.layers.Conv2DTranspose(filters, kernel_size, strides=(2, 2), padding=padding)
        self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size, activation=activation, padding=padding)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size, activation=activation, padding=padding)
        self.bn2 = tf.keras.layers.BatchNormalization()

    def call(self, inputs, concat):
        x = self.up_conv(inputs)
        x = tf.concat([x, concat], axis=-1)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        return x

In [18]:
class UNet(tf.keras.Model):
    def __init__(self, filters, num_classes, kernel_size=(3, 3), activation='relu', padding='same', **kwargs):
        super(UNet, self).__init__(**kwargs)
        self.encoder1 = UNetEncoder(filters, kernel_size, activation, padding)
        self.encoder2 = UNetEncoder(filters*2, kernel_size, activation, padding)
        # self.encoder3 = UNetEncoder(filters*4, kernel_size, activation, padding)
        # self.encoder4 = UNetEncoder(filters*8, kernel_size, activation, padding)
        self.center = tf.keras.layers.Conv2D(filters*16, kernel_size, activation=activation, padding=padding)
        # self.decoder4 = UNetDecoder(filters*8, kernel_size, activation, padding)
        # self.decoder3 = UNetDecoder(filters*4, kernel_size, activation, padding)
        self.decoder2 = UNetDecoder(filters*2, kernel_size, activation, padding)
        self.decoder1 = UNetDecoder(filters, kernel_size, activation, padding)
        self.output_layer = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')

    def call(self, inputs):
        encoder1, pool1 = self.encoder1(inputs)
        encoder2, pool2 = self.encoder2(pool1)
        # encoder3, pool3 = self.encoder3(pool2)
        # encoder4, pool4 = self.encoder4(pool3)
        center = self.center(pool2)
        # decoder4 = self.decoder4(center, encoder4)
        # decoder3 = self.decoder3(decoder4, encoder3)
        decoder2 = self.decoder2(center, encoder2)
        decoder1 = self.decoder1(decoder2, encoder1)
        output = self.output_layer(decoder1)
        return output

In [19]:
unet = UNet(64, 2)

#train the model
unet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [20]:
image, label = next(iter(data))

In [21]:
unet.fit(image, label, epochs=10)

Epoch 1/10


2023-06-02 21:01:21.454051: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 268435456 exceeds 10% of free system memory.
2023-06-02 21:01:24.093063: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 268435456 exceeds 10% of free system memory.
2023-06-02 21:01:24.096545: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 301989888 exceeds 10% of free system memory.
2023-06-02 21:01:24.096638: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 301989888 exceeds 10% of free system memory.


In [13]:
image.shape

TensorShape([8, 256, 256, 3])